# Backtest Analysis Notebook

This notebook provides visualizations and analysis of backtest results.

**Features:**
- Load backtest artifacts (trades, equity curve, summary)
- Visualize equity curve and drawdowns
- Analyze trade distribution and P&L
- Calculate additional performance metrics
- Export visualizations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

print("✅ Imports successful")

## 1. Load Backtest Results

Specify the run name or leave as default to load the latest run.

In [ ]:
# Configuration
ARTIFACTS_DIR = '../artifacts'  # Adjust path if needed
RUN_NAME = None  # Set to specific run name or None for latest

# Find run files
artifacts_path = Path(ARTIFACTS_DIR)

if RUN_NAME is None:
    # Find latest summary file
    summary_files = sorted(artifacts_path.glob('*_summary.json'))
    if not summary_files:
        raise FileNotFoundError(f"No backtest results found in {ARTIFACTS_DIR}")
    latest_summary = summary_files[-1]
    RUN_NAME = latest_summary.stem.replace('_summary', '')
    print(f"Loading latest run: {RUN_NAME}")
else:
    print(f"Loading run: {RUN_NAME}")

# Load files
summary_file = artifacts_path / f"{RUN_NAME}_summary.json"
trades_file = artifacts_path / f"{RUN_NAME}_trades.csv"
equity_file = artifacts_path / f"{RUN_NAME}_equity_curve.csv"
rejected_file = artifacts_path / f"{RUN_NAME}_rejected_trades.csv"

# Load data
with open(summary_file) as f:
    summary = json.load(f)

trades_df = pd.read_csv(trades_file, parse_dates=['entry_time', 'exit_time'])
equity_df = pd.read_csv(equity_file)

if rejected_file.exists():
    rejected_df = pd.read_csv(rejected_file, parse_dates=['timestamp'])
    print(f"✅ Loaded {len(rejected_df)} rejected trades")
else:
    rejected_df = pd.DataFrame()

print(f"✅ Loaded {len(trades_df)} trades")
print(f"✅ Loaded equity curve with {len(equity_df)} points")

## 2. Summary Statistics

In [ ]:
# Display summary metrics
metrics = summary['metrics']

print("\n📊 BACKTEST SUMMARY")
print("=" * 50)
print(f"Initial Capital: ${summary['config']['initial_capital']:,.2f}")
print(f"Total Trades: {metrics['total_trades']}")
print(f"Rejected Trades: {summary['rejected_count']}")
print(f"\nPerformance:")
print(f"  Total P&L: ${metrics['total_pnl']:,.2f}")
print(f"  Total Return: {metrics['total_return']:.2f}%")
print(f"  Win Rate: {metrics['win_rate']:.2f}%")
print(f"  Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"  Max Drawdown: {metrics['max_drawdown_pct']:.2f}%")
print(f"  Profit Factor: {metrics['profit_factor']:.2f}")
print(f"\nTrade Statistics:")
print(f"  Avg Trade: ${metrics['avg_trade']:.2f}")
print(f"  Avg Win: ${metrics['avg_win']:.2f}")
print(f"  Avg Loss: ${metrics['avg_loss']:.2f}")
print(f"  Best Trade: ${metrics['best_trade']:.2f}")
print(f"  Worst Trade: ${metrics['worst_trade']:.2f}")

## 3. Equity Curve Visualization

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Equity curve
ax1.plot(equity_df.index, equity_df['equity'], linewidth=2, color='blue')
ax1.axhline(y=summary['config']['initial_capital'], color='gray', linestyle='--', alpha=0.5, label='Initial')
ax1.set_ylabel('Equity ($)', fontsize=12)
ax1.set_title('Equity Curve', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Drawdown
peak = equity_df['equity'].expanding(min_periods=1).max()
drawdown = (equity_df['equity'] - peak) / peak * 100
ax2.fill_between(drawdown.index, drawdown, 0, color='red', alpha=0.3)
ax2.set_xlabel('Trade Number', fontsize=12)
ax2.set_ylabel('Drawdown (%)', fontsize=12)
ax2.set_title('Drawdown', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(artifacts_path / f"{RUN_NAME}_equity_curve.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Saved chart to {RUN_NAME}_equity_curve.png")

## 4. Trade Distribution Analysis

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# P&L distribution
axes[0, 0].hist(trades_df['pnl'], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
axes[0, 0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('P&L ($)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('P&L Distribution', fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Trade duration
axes[0, 1].hist(trades_df['duration_hours'], bins=30, color='green', edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Duration (hours)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Trade Duration Distribution', fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Side distribution
side_counts = trades_df['side'].value_counts()
axes[1, 0].bar(side_counts.index, side_counts.values, color=['lightcoral', 'lightgreen'])
axes[1, 0].set_xlabel('Side')
axes[1, 0].set_ylabel('Count')
axes[1, 0].set_title('Trade Side Distribution', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Exit reason
exit_counts = trades_df['exit_reason'].value_counts()
axes[1, 1].bar(exit_counts.index, exit_counts.values, color='orange')
axes[1, 1].set_xlabel('Exit Reason')
axes[1, 1].set_ylabel('Count')
axes[1, 1].set_title('Exit Reason Distribution', fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(artifacts_path / f"{RUN_NAME}_distributions.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Saved chart to {RUN_NAME}_distributions.png")

## 5. Cumulative P&L Over Time

In [ ]:
trades_df['cumulative_pnl'] = trades_df['pnl'].cumsum()

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(trades_df.index, trades_df['cumulative_pnl'], linewidth=2, color='purple')
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.fill_between(trades_df.index, trades_df['cumulative_pnl'], 0, 
                where=(trades_df['cumulative_pnl'] >= 0), 
                color='green', alpha=0.2, label='Profit')
ax.fill_between(trades_df.index, trades_df['cumulative_pnl'], 0, 
                where=(trades_df['cumulative_pnl'] < 0), 
                color='red', alpha=0.2, label='Loss')
ax.set_xlabel('Trade Number', fontsize=12)
ax.set_ylabel('Cumulative P&L ($)', fontsize=12)
ax.set_title('Cumulative P&L Over Time', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(artifacts_path / f"{RUN_NAME}_cumulative_pnl.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Saved chart to {RUN_NAME}_cumulative_pnl.png")

## 6. Trade Performance by Side

In [ ]:
# Calculate metrics by side
side_metrics = trades_df.groupby('side').agg({
    'pnl': ['count', 'sum', 'mean', 'std'],
    'pnl_pct': 'mean',
    'duration_hours': 'mean'
}).round(2)

# Win rate by side
win_rate_by_side = trades_df.groupby('side').apply(
    lambda x: (x['pnl'] > 0).sum() / len(x) * 100
).round(2)

print("\n📊 PERFORMANCE BY SIDE")
print("=" * 50)
print(side_metrics)
print(f"\nWin Rate by Side:")
for side, rate in win_rate_by_side.items():
    print(f"  {side}: {rate:.2f}%")

## 7. Monthly Performance (if applicable)

In [ ]:
if len(trades_df) > 0:
    trades_df['month'] = trades_df['exit_time'].dt.to_period('M')
    monthly_pnl = trades_df.groupby('month')['pnl'].sum()
    
    if len(monthly_pnl) > 1:
        fig, ax = plt.subplots(figsize=(14, 6))
        colors = ['green' if x >= 0 else 'red' for x in monthly_pnl.values]
        ax.bar(range(len(monthly_pnl)), monthly_pnl.values, color=colors, alpha=0.7)
        ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
        ax.set_xlabel('Month', fontsize=12)
        ax.set_ylabel('P&L ($)', fontsize=12)
        ax.set_title('Monthly P&L', fontsize=14, fontweight='bold')
        ax.set_xticks(range(len(monthly_pnl)))
        ax.set_xticklabels([str(m) for m in monthly_pnl.index], rotation=45)
        ax.grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.savefig(artifacts_path / f"{RUN_NAME}_monthly_pnl.png", dpi=150, bbox_inches='tight')
        plt.show()
        
        print(f"✅ Saved chart to {RUN_NAME}_monthly_pnl.png")
    else:
        print("⚠️ Not enough data for monthly analysis")
else:
    print("⚠️ No trades to analyze")

## 8. Export Summary Report

In [ ]:
report = f"""
BACKTEST ANALYSIS REPORT
========================
Run: {RUN_NAME}
Timestamp: {summary['timestamp']}

CONFIGURATION
-------------
Initial Capital: ${summary['config']['initial_capital']:,.2f}
Position Size: {summary['config']['position_size_pct']*100:.1f}%
Leverage: {summary['config']['leverage']}x
Maker Fee: {summary['config']['maker_fee']*100:.3f}%
Taker Fee: {summary['config']['taker_fee']*100:.3f}%
Stop Loss: {summary['config']['stop_loss_pct']*100:.1f}%
Take Profit: {summary['config']['take_profit_pct']*100:.1f}%

PERFORMANCE METRICS
-------------------
Total Trades: {metrics['total_trades']}
Rejected Trades: {summary['rejected_count']}
Total P&L: ${metrics['total_pnl']:,.2f}
Total Return: {metrics['total_return']:.2f}%
Win Rate: {metrics['win_rate']:.2f}%
Sharpe Ratio: {metrics['sharpe_ratio']:.2f}
Sortino Ratio: {metrics['sortino_ratio']:.2f}
Max Drawdown: {metrics['max_drawdown_pct']:.2f}%
Profit Factor: {metrics['profit_factor']:.2f}

TRADE STATISTICS
----------------
Average Trade: ${metrics['avg_trade']:.2f}
Average Win: ${metrics['avg_win']:.2f}
Average Loss: ${metrics['avg_loss']:.2f}
Best Trade: ${metrics['best_trade']:.2f}
Worst Trade: ${metrics['worst_trade']:.2f}
Average Duration: {metrics['avg_trade_duration_hours']:.2f} hours

ANALYSIS
--------
This backtest was run with the Yun Min trading system.
All artifacts and visualizations have been saved to: {artifacts_path}
"""

report_file = artifacts_path / f"{RUN_NAME}_report.txt"
with open(report_file, 'w') as f:
    f.write(report)

print(report)
print(f"\n✅ Report saved to {report_file}")